In [ ]:
import pandas as pd
data = pd.read_csv('processed_datascience.csv.bz2')
import numpy as np
from calzone import describe
from feature_extraction import Blob
from sklearn.base import BaseEstimator, TransformerMixin
from textblob import TextBlob
from textblob.taggers import NLTKTagger, PatternTagger
from collections import Counter
from itertools import chain
from nltk.corpus import cmudict
from textstat.textstat import textstat


import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import Blob, Words, Exclude, WordCount, POS, Readable
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.externals import joblib
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer

data = pd.read_csv('processed_datascience.csv.bz2')
data['z_scores'] = np.abs((data.ups-data.ups.mean())/data.ups.std())
data = data[data['z_scores']<= 2.5]

## Optional: Log transformation of up-votes
data['log_ups'] = np.log1p(data['ups'])

# Create Label column defining whether or not the article's upvotes exceed the average vote for the subreddit
data['gtavg'] = data.log_ups > data.log_ups.mean()
train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.gtavg, 
                                                    test_size=0.20,
                                                    random_state=25)

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
index = 3
def lemmatize(token, tag):
    if tag[0].lower() in ['n', 'v']:
        return lemmatizer.lemmatize(token, tag[0].lower())
    return token
tagged_corpus = [pos_tag(word_tokenize(data['title'][index].replace("/"," " )))]
lc = [lemmatize(token, tag) for token, tag in tagged_corpus[0]]
tc = [stemmer.stem(token) for token in word_tokenize(data.title[index].replace("/", " "))]
print(data.title[index])
print(tc)
print(lc)
# lemma = [[lemmatize(token, tag) for token, tag in document ] for document in tagged_corpus]
# [lemmatizer.lemmatize(token) for token, _ in tc]
#lemmatize('analytics', 'v')

In [ ]:
def lemmatize(data):
    def lemma(token, tag):
        if tag[0].lower() in ['n', 'v']:
            return lemmatizer.lemmatize(token, tag[0].lower())
        return token

    lemmatizer = WordNetLemmatizer()
    tagged_corpus = [pos_tag(word_tokenize(title)) for title in data['title'].str.replace("/", " ").str.lower()]
    lc = [" ".join([lemma(token, tag) for token, tag in title]) for title in tagged_corpus]
    return lc

In [ ]:
a = lemmatize(data)

In [ ]:
a

In [ ]:
tfidf = TfidfVectorizer(token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', analyzer='word', ngram_range=(1,3), stop_words='english', strip_accents='unicode').fit(train_X, train_y)
train_titles_tfidf = tfidf.transform(train_X)
test_titles_tfidf = tfidf.transform(test_X)

In [ ]:
svd = TruncatedSVD(n_components=120)
svd.fit(train_titles_tfidf)
svd.get_params()

In [ ]:
#train_titles_tfidf = tfidf.transform(train_X)
#test_titles_tfidf = tfidf.transform(test_X)

# reg = GradientBoostingClassifier()
# reg.fit(train_titles_tfidf, train_labels)
# print(reg.score(test_titles_tfidf, test_labels))
# train_predictions = reg.predict(train_titles_tfidf)
# test_predictions = reg.predict(test_titles_tfidf)
# print(mean_absolute_error(test_predictions, test_labels))

# # print(train_set.iloc[0])
# print(train_predictions[3], train_labels.iloc[3])
# plt.scatter(train_labels, train_predictions)
# plt.show()

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('pipe', Pipeline([
                ('inner', FeatureUnion(
                    transformer_list=[
                        ('pos', POS()),

                        ('read', Readable()),

                        ('words', Words()),

                        ('blob', Pipeline([
                            ('all', Blob()),
                            ('minmax', MinMaxScaler()),
                        ])),
                ])),
                ('select', SelectFromModel(ExtraTreesClassifier()))
                     
            ])),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', ngram_range=(1,3), sublinear_tf=True, stop_words='english')),
                ('svd', TruncatedSVD(n_components=120)),
                ('normalize', MinMaxScaler(copy=False)),
                ('selector', SelectPercentile(f_classif, percentile=10))
            ])),

            
            ])),
    ('clf', RandomForestClassifier(n_estimators=190, n_jobs=-1, max_depth=5, max_features='log2',
                                   min_samples_leaf=1, min_samples_split=77)),
        ])

In [ ]:
pipeline.fit(train_X, train_y)

In [ ]:
a = fu.fit_transform(train_X)
b = fu.transform(test_X)

In [ ]:
clf = ExtraTreesClassifier()
clf = clf.fit(a, train_y)
a.shape

In [ ]:
clf.feature_importances_
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(a)
X_new.shape

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('pos', POS()),
            
            ('read', Readable()),
            
            ('words', Words()),
                                
            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
            
            ])),
    ('select', SelectFromModel(ExtraTreesClassifier())),
        ])

# Train model
a = pipeline.fit(train_X, train_y)

In [ ]:
b = a.transform(train_X)

In [ ]:
b.shape

In [ ]:
fu = FeatureUnion(
        transformer_list=[
            ('pos', POS()),

            ('read', Readable()),

            ('words', Words()),

            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
    ])

In [ ]:
fu.fit(train_X, train_y)

In [ ]:
a = fu.transform(train_X)

In [ ]:
a.shape

In [ ]:
x = Words()
y = x.transform(train_X)

In [ ]:
len(y[1])